In [ ]:
import numpy as np
from dadk.QUBOSolverCPU import *
from dadk.BinPol import *
from datetime import datetime
import pandas as pd

# Input Data

In [ ]:
# Assets
asset_quantity = pd.read_csv("data/sample_asset_quantity.csv", header=None)[0].values.tolist()
asset_value = pd.read_csv("data/sample_asset_value.csv", header=None)[0].values.tolist()
asset_tiers = pd.read_csv("data/sample_asset_tiers.csv", header=None)[0].values.tolist()

n_assets = len(asset_quantity)

# Accounts 
account_exposure = pd.read_csv("data/sample_account_exposure.csv", header=None)[0].values.tolist()
account_duration = pd.read_csv("data/sample_account_duration.csv", header=None)[0].values.tolist()

n_accounts = len(account_exposure)

# Haircuts 
haircuts = (pd.read_csv("data/sample_haircuts.csv", header=None).values.tolist())

#Single Limits
#single_limits = (pd.read_csv("data/sample_single_limits.csv", header=None).values.tolist())

# Updated Tier list 
cost_factor_matrix = np.zeros(shape=(n_assets, n_accounts))

for i in range(n_assets):
    for j in range(n_accounts):
        cost_factor_matrix[i,j] = abs(account_duration[j] - asset_tiers[i])

## Penalty Terms

In [ ]:
# Penalty terms
lambda_cost_fn = 1e2
lambda_consistency = 5e5
lambda_exposure = 1
#lambda_single_lim = 1

# Defining the QUBO

In [ ]:
# n-bit binary variables
n_binary = 7
max_binary_value = 2**(n_binary) - 1 

binary_coeff_vector = np.zeros(n_binary)

for i in range(n_binary):
    binary_coeff_vector[i] = 2**i

# Binary_variables
binary_variables = VarShapeSet(BitArrayShape(name='bin_vars', shape=(n_assets, n_accounts, n_binary), axis_names=['Assets', 'Accounts', 'Binary']))



consistency_slack_var = VarShapeSet(BitArrayShape(name='consistency_slack_var', shape=(n_assets, (int(np.ceil(np.log2(max_binary_value))))), axis_names=['Assets', 'Slack']))

#Variables for single limits

# num_slack_single_limit_arr = np.zeros(shape=(n_assets, n_accounts))

# for i in range(n_assets):
#     for j in range(n_accounts):
#         try:
#             num_slack_single_limit_arr[i][j] = int(np.ceil(np.log2((single_limits[i][j]/asset_quantity[i])*max_binary_value)))
#         except OverflowError:
#             print("pass")


# single_lim_variables = []
# for i in range(n_assets):
#     for j in range(n_accounts):
#         if num_slack_single_limit_arr[i][j] != 0:
#             single_lim_variables.append(BitArrayShape(name=f"singlelim_slack_asset{i+1}_account{j+1}", 
#                                                                    shape=(1, num_slack_single_limit_arr[i][j]), axis_names=['1', 'Single lim']))

# single_lim_slack_variables = VarShapeSet(*single_lim_variables)




# Constructing the QUBO equation
This involves the QUBO objective function, the consistency contraints, the exposure constraints.

In [ ]:
# QUBO objective function 
cost_function = BinPol(binary_variables)
for i in range(n_assets):
    for j in range(n_accounts):
        for k in range(n_binary):
            cost_function.add_term(cost_factor_matrix[i][j]*binary_coeff_vector[k]/max_binary_value,(("bin_vars", i, j, k),))

In [ ]:
# Consistency constraint 
con_constraint = BinPol(consistency_slack_var)

for i in range(n_assets):
    aux_term = BinPol(binary_variables)
    consistency_penalty_slack = BinPol(consistency_slack_var)
    #slack
    for l in range((int(np.ceil(np.log2(max_binary_value))))):
        consistency_penalty_slack.add_term(2**l,(('consistency_slack_var', i,l)))
    #decision variable
    for j in range(n_accounts):
        for k in range(n_binary):
            aux_term.add_term(binary_coeff_vector[k],(("bin_vars", i, j, k),))
    aux_term.add_term(-max_binary_value,()).add(consistency_penalty_slack)
    aux_term.power(2)
    con_constraint.add(aux_term)
#pprint(con_constraint)


In [ ]:
#Exposure constraint
exposure_penalty_term = BinPol(binary_variables)

for j in range(n_accounts):
    aux_term1 = BinPol(binary_variables)
    for i in range(n_assets):
        for k in range(n_binary):
            aux_term1.add_term(binary_coeff_vector[k]*asset_quantity[i]*asset_value[i]*haircuts[i][j]/(100*max_binary_value),(("bin_vars", i, j, k),))
    aux_term1.add_term(-account_exposure[j],())
    aux_term1.power(2)
    exposure_penalty_term.add(aux_term1)

#pprint(exposure_penalty_term)

In [ ]:
# # Single limit constraint

# #slack part

# single_limit_penalty_slack = []
# for i in range(n_assets):
#     aux_array = []
#     for j in range(n_accounts):
#         aux_term = BinPol(var_shape_set=single_lim_slack_variables)
#         for l in range(int(num_slack_single_limit_arr[i][j])):
#             aux_term.add_term(2**l,((f"singlelim_slack_asset{i+1}_account{j+1}",0,l),))
#         aux_array.append(aux_term)
#     single_limit_penalty_slack.append(aux_array)

# #decision variable part

# single_limit_penalty_term = BinPol(binary_variables)
# #This for loop works in conjuction with the nontraditional way of creating slack variables
# for i in range(n_assets):
#     for j in range(n_accounts):
#         aux_term1 = BinPol(binary_variables)
#         for k in range(n_binary):
#             aux_term1.add_term(binary_coeff_vector[k],(('bin_vars',i,j,k),))
#         aux_term1.add_term(-single_limits[i][j]*max_binary_value/asset_quantity[i])
#         aux_term1.add(single_limit_penalty_slack[i][j])
#         single_limit_penalty_term.add(aux_term1**2)

In [ ]:
# Final QUBO
QUBO_equation = lambda_cost_fn*cost_function + lambda_consistency*con_constraint  + lambda_exposure*exposure_penalty_term #+lambda_single_lim*single_limit_penalty_term

# Run Solver

In [ ]:
# Define solver parameters
solver = QUBOSolverCPU(
   
    number_iterations    = 1000,               # total number of itrations per run
    number_runs          = 1000,                 # number of stochastically independant runs
    temperature_start    = 5000,               # start temperature for annealing as float value
    temperature_end      = 10,                 # end temperature for annealing as float value 
    temperature_mode     = 0,                         # 0: reduce temperature by factor (1-temperature_decay) every temperature_interval steps
                                                      # 1: reduce temperature by factor (1-temperature_decay*temperature) every temperature_interval steps
                                                      # 2: reduce temperature by factor (1-temperature_decay*temperature^2) every temperature_interval steps
    temperature_decay    = 0.0095,             # see temperature_mode 0
    temperature_interval = 1,                  # see temperature_mode 0
    offset_increase_rate = 5000.0,             # increase of dynamic offset when no bit selected, set to 0.0 to switch off dynamic offset
    graphics             = True                # create data for graphics output
)

In [ ]:
# Run Solver
start=datetime.now()
solution_list = solver.minimize(QUBO_equation)

#Get results
solution = solution_list.get_minimum_energy_solution()
configuration = solution.configuration
my_bit_array = solution.extract_bit_array("bin_vars")

# Verify Results

In [ ]:
bitstring = ''

exposure_solution = np.zeros(shape = (n_accounts))
allocation_solution = np.zeros(shape = (n_assets))

print("=====Each Individual Allocation Percentage=====")

for i in range(n_assets):
    asset = 0
    for j in range(n_accounts):
        bit = ''
        decimal = 0
        for k in range(n_binary):
            bit += str(my_bit_array[i][j][k])
            decimal +=(2**k)*my_bit_array[i][j][k]
        asset += decimal*(100/max_binary_value)
        print(f"Assign {decimal*(100/max_binary_value):0.2f}% of asset {i+1} to account {j+1}")
        exposure_solution[j] += (decimal /max_binary_value) * asset_value[i] * asset_quantity[i]*haircuts[i][j]/100
    allocation_solution[i] = asset
    

In [ ]:
print("====Validation for Consistency====")
for i in range(n_assets):
    print(f"{allocation_solution[i]:0.2f}% of asset {i+1} posted")

In [ ]:
print("=====Validation for Exposure======")
for i in range(n_accounts):
    print(f"${exposure_solution[i]:,} of collateral posted, ${account_exposure[i]:,} required")

In [ ]:
#print("===Validation of Single limits =====")
# for i in range(n_assets):
#     for j in range(n_accounts):
#         decimal = 0
#         for k in range(n_binary):
#             decimal +=(2**k)*my_bit_array[i][j][k]
#         allocation = (decimal/max_binary_value)*asset_quantity[i] 
#         print(f"Amount allocated: {allocation:0.2f}. Single Limit: {single_limits[i][j]:0.2f}")
#         print(f"Constraint Satisfied?: {allocation <= single_limits[i][j]}")
        

In [ ]:
print(f"The objective function is: {cost_function.compute(configuration)}")
print("The optimal objective function is: 0.4299816571359823")

In [ ]:
# Print runtime
print(datetime.now()-start)

# Export to .csv

In [ ]:
# Export to .csv
Q_value_balanced_fujitsu = np.zeros(shape=(n_assets,n_accounts))


for i in range(n_assets):
    asset = 0
    for j in range(n_accounts):
        bit = ''
        decimal = 0
        for k in range(n_binary):
            bit += str(my_bit_array[i][j][k])
            decimal +=binary_coeff_vector[k]*my_bit_array[i][j][k]
        asset += decimal*(100/max_binary_value)
        Q_value_balanced_fujitsu[i][j] = decimal/max_binary_value

print(Q_value_balanced_fujitsu)

#Uncomment the line below to save the Data to csv

# pd.DataFrame(Q_value_balanced_fujitsu).to_csv("Q_value_balanced_fujitsu_no_single_lim_trial.csv", header=None, index=None)